In [14]:
import stim
import numpy as np
from IPython.display import SVG
import sys
import os
sys.path.append(os.path.abspath(".."))  # add project root
from src import distance_d_repetition_code, collect_stats





# One detailed run
out = distance_d_repetition_code(distance=9, rounds=20, noise=0.05)
display(out["SVG"])
print(out["raw_logical"], out["corrected_logical"])


ImportError: cannot import name 'distance_d_repetition_code' from 'src' (c:\Users\msokh\Quantum-Error-Correction\src\__init__.py)